In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
def get_urls(page):
    urls=[]
    response = requests.get(page)
    soup = BeautifulSoup(response.text, 'html.parser')
    for url in soup.findAll('a'):
        if '/auctions/auction' in str(url.get('href')):
            urls.append('https://www.phillips.com'+str(url.get('href')))
    return(set(urls))

def aut(item):
    try:
        return(item.findAll('p',{"class": "phillips-lot__description__artist"})[0]['title'])
    except:
        return np.nan
    
def ptg(item):
    try:
        return(item.findAll('p',{"class": "phillips-lot__description__title"})[0]['title'])
    except:
        return np.nan
    
def lw(item):
    try:
        return(item.findAll('p',{"class": "phillips-lot__description__estimate"})[0].text.split('\xa0')[1])
    except:
        return np.nan    
    
def hgh(item):
    try:
        return(item.findAll('p',{"class": "phillips-lot__description__estimate"})[0].text.split('\xa0')[3])
    except:
        return np.nan   
    
def prc(item):
    try:
        return(item.findAll('p',{"class": "phillips-lot__sold"})[0].text.split()[-1])
    except:
        return np.nan   
    
def get_info_page(page,author,painting,price,low_estimate,high_estimate):
    response = requests.get(page)
    soup = BeautifulSoup(response.text, 'html.parser')
    lot=soup.findAll('div',{"class": "phillips-lot"})
    for item in lot:
        author.append(aut(item))
        painting.append(ptg(item))
        price.append(prc(item))
        low_estimate.append(lw(item))
        high_estimate.append(hgh(item))
    return()

In [3]:
author=[]
painting=[]
price=[]
low_estimate=[]
high_estimate=[]

urls=get_urls('https://www.phillips.com/auctions/past')

In [4]:
for url in tqdm(urls):
    get_info_page(url,author,painting,price,low_estimate,high_estimate)

100%|██████████| 569/569 [15:50<00:00,  1.67s/it]


In [5]:
x=pd.DataFrame([author,painting,price,low_estimate,high_estimate]).T
x.columns=['author','painting','price','lw','hg']

In [8]:
x.isna().sum()

author       2610
painting      838
price       23346
lw           1462
hg           1462
dtype: int64

In [11]:
x[x.painting=='A Cultured Pearl and Diamond Suite']

,author,painting,price,lw,hg
0,NaN,A Cultured Pearl and Diamond Suite,"$3,500","$1,500","2,500"
14856,NaN,A Cultured Pearl and Diamond Suite,"$18,750","$15,000","20,000"
31517,NaN,A Cultured Pearl and Diamond Suite,NaN,"$10,000","15,000"


In [16]:
x.shape

(85607, 5)

In [14]:
x[x.author.isna()]

,author,painting,price,lw,hg
0,NaN,A Cultured Pearl and Diamond Suite,"$3,500","$1,500","2,500"
1,NaN,A Gold and Diamond Cuff Bracelet,"$2,750","$2,000","3,000"
2,NaN,A Gold and Diamond Necklace,"$3,750","$1,800","2,500"
8,NaN,A Baroque Cultured Pearl and Colored Diamond N...,NaN,"$8,000","12,000"
9,NaN,A Gold and Gem-Set Pendant Necklace,"$8,750","$3,000","5,000"
...,...,...,...,...,...
85160,NaN,NaN,NaN,NaN,NaN
85163,NaN,NaN,NaN,NaN,NaN
85377,NaN,A Colored Diamond and Diamond Necklace,"CHF74,500","CHF65,000","80,000"
85467,NaN,NaN,NaN,NaN,NaN


In [19]:
x[x.author=='Gio Ponti']

,author,painting,price,lw,hg
1671,Gio Ponti,‘Figurina’,NaN,"£8,000","12,000"
1678,Gio Ponti,Early cabinet,"£22,500","£12,000","18,000"
1682,Gio Ponti,Daybed,"£10,000","£8,000","12,000"
1683,Gio Ponti,‘Four Seasons and Sirens’ chandelier,"£11,250","£7,000","9,000"
1758,Gio Ponti,Integrated sideboard and bookcase,"£86,500","£15,000","20,000"
...,...,...,...,...,...
85097,Gio Ponti,Unique and important coffee table,"£86,500","£30,000","50,000"
85109,Gio Ponti,"Rare bench, designed for the Antica Quadreria,...","£158,500","£35,000","45,000"
85110,Gio Ponti,Early ceiling light,"£22,500","£10,000","15,000"
85562,Gio Ponti,Set of six early lights,"$38,750","$8,000","12,000"
